# Close approach

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [28]:
import numpy as np
from numpy.linalg import norm

# import plotting modules
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.widgets as mw  # get access to the widgets

%matplotlib qt

from galaxy.db import DB
from galaxy.galaxies import Galaxies
from galaxy.galaxy import Galaxy
from galaxy.timecourse import TimeCourse
from galaxy.plots import Plots
from galaxy.approaches import Approaches

In [2]:
import mpl_scatter_density

# Make the norm object to define the image stretch
from astropy.visualization import LogStretch
from astropy.visualization.mpl_normalize import ImageNormalize
lognorm = ImageNormalize(vmin=0., vmax=1000, stretch=LogStretch())

In [3]:
#adjust tick label font size
label_size = 14
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

In [29]:
def plot_density_views(disks, title, xlim=150, ylim=150, pngout=False, fname=None):
    fig = plt.figure(figsize=(10,10))
    fontsize = 16

    # top left
    ax0 = fig.add_subplot(1, 1, 1, projection='scatter_density')
    ax0.scatter_density(disks[0], disks[1], norm=lognorm)

    ax0.set_xlim(-xlim, xlim)
    ax0.set_ylim(-ylim, ylim)

    # Add axis labels (standard pyplot)
    ax0.set_xlabel('x (kpc)', fontsize=fontsize)
    ax0.set_ylabel('y (kpc)', fontsize=fontsize)
    
    # Save file
    if pngout:
        plt.savefig(fname, dpi='figure')
        plt.close();  

In [6]:
snap = 301
disks, t = get_xyz(snap)
title = f"\n\nSnap: {snap:03}\nElapsed time: {t:5.3f} Gyr"

In [7]:
plot_density_views(disks, title, pngout=False, fname=f"approach_{snap:03}.png")
print(snap, end=' ')

301 

/home/colin/.pyenv/versions/miniconda3-4.2.12/envs/400b/lib/python3.6/site-packages/mpl_scatter_density/generic_density_artist.py:74: RuntimeWarning: All-NaN slice encountered
  vmin = self._density_vmin(array)
/home/colin/.pyenv/versions/miniconda3-4.2.12/envs/400b/lib/python3.6/site-packages/mpl_scatter_density/generic_density_artist.py:79: RuntimeWarning: All-NaN slice encountered
  vmax = self._density_vmax(array)


In [62]:
snap = 300
app = Approaches(snap=snap, usesql=True)
t = app.time.value / 1000

In [64]:
# get two (3,N) arrays with just position/velocity coordinates
xyz = app.xyz()
vxyz = app.vxyz()

v = norm(vxyz, axis=0)

# center the collection visually (not CoM)
centroid = np.mean(xyz, axis=1)
xyz -= centroid[:,np.newaxis]
xyz.shape, v.shape

((3, 1166500), (1166500,))

In [168]:
index = None

def callbackRectangle( click, release ): # the events are click and release
    """
    """
    
    global index
    
    # create a rectangle
    width = np.abs(release.xdata - click.xdata)
    height = np.abs(release.ydata - click.ydata)
    
    rect = plt.Rectangle( (click.xdata, click.ydata), width, height,
                            fill=False, color='yellow', lw=1)
    
    # clear old rectangles, add new one
    [p.remove() for p in reversed(ax0.patches)]
    ax0.add_patch(rect)
    
    # extrema will be useful for setting axes
    xmin = min([click.xdata, release.xdata])
    xmax = max([click.xdata, release.xdata])
    ymin = min([click.ydata, release.ydata])
    ymax = max([click.ydata, release.ydata])
    
    index = np.where( (x > xmin) & (x < xmax) & (y > ymin) & (y < ymax) )
    
    # fill in mid and right panels
    add_velocity_plot(ax1, xmin, xmax, ymin, ymax)
    add_origin_plot(ax2, xmin, xmax, ymin, ymax)
    plt.tight_layout()
    
#     ref = ax[1].scatter(x[index], y[index], s=1)

    # Save the file

In [162]:
def onKeyPressed(event):
    
    # eventually want to reset by removing selection
    pass

#     if event.key in ['R', 'r']:
#         ax.set_xlim(-30,30)
#         ax.set_ylim(-30,30)

In [203]:
def add_velocity_plot(ax, xmin, xmax, ymin, ymax):

    sel_inx = index[0]
    sel_xyz = xyz[:,sel_inx]
    sel_vxyz = vxyz[:,sel_inx]
#     print('x: ', len(selected['x']), selected['x'])

    if index is not None:
#         ax1.quiver(x[index], y[index], vx[index], vy[index], color='gray')
        ax1.quiver(sel_xyz[0], sel_xyz[1], sel_vxyz[0], sel_vxyz[1], color='gray')

    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)

    # Add axis labels (standard pyplot)
    ax.set_xlabel('x (kpc)', fontsize=fontsize)
    ax.set_ylabel('y (kpc)', fontsize=fontsize)

In [230]:
def add_origin_plot(ax, xmin, xmax, ymin, ymax):

    if index is not None:
        sel_inx = index[0]
        sel_x, sel_y, sel_z = xyz[:,sel_inx]
        sel_data = app.data[sel_inx]
        sel_galname = sel_data['galname']
        sel_type = sel_data['type']
        
        markersize = 5
        group = np.where((sel_galname=='MW ') & (sel_type==2))
        ax.scatter(sel_x[group], sel_y[group], color='k', s=markersize, label='MW disk')
        
        group = np.where((sel_galname=='M31') & (sel_type==2))
        ax.scatter(sel_x[group], sel_y[group], color='g', s=markersize, label='M31 disk')
        
        group = np.where((sel_galname=='MW ') & (sel_type==3))
        ax.scatter(sel_x[group], sel_y[group], color='r', s=markersize, marker='^', label='MW bulge')
        
        group = np.where((sel_galname=='M31') & (sel_type==3))
        ax.scatter(sel_x[group], sel_y[group], color='b', s=markersize, marker='^', label='M31 bulge')

    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)

    # Add axis labels (standard pyplot)
    ax.set_xlabel('x (kpc)', fontsize=fontsize)
    ax.set_ylabel('y (kpc)', fontsize=fontsize)
    
    if ax.get_legend() is not None:
        ax.get_legend().remove()
    ax.legend(shadow=True)

In [231]:
x,y,z = xyz
vx,vy,vz = vxyz

fig = plt.figure(figsize=(18,6))
fontsize = 16
titlesize = 20

xlim = ylim = 150

# top left
ax0 = fig.add_subplot(1, 3, 1, projection='scatter_density')
ax0.scatter_density(x, y, norm=lognorm)

ax0.set_xlim(-xlim, xlim)
ax0.set_ylim(-ylim, ylim)

# Add axis labels (standard pyplot)
ax0.set_xlabel('x (kpc)', fontsize=fontsize)
ax0.set_ylabel('y (kpc)', fontsize=fontsize)

ax0.set_title(f"Density at t={t:5.2f} Gyr", fontsize=titlesize)

ax1 = fig.add_subplot(1, 3, 2)
ax1.set_title('Velocities of selected stars', fontsize=titlesize)

ax2 = fig.add_subplot(1, 3, 3)
ax2.set_title('Origins of selected stars', fontsize=titlesize)



rs = mw.RectangleSelector( ax0,                        # the axes to attach to
                           callbackRectangle,         # the callback function
                           drawtype='box',            # draw a box when selecting a region
                           button=[1, 3],             # allow us to use left or right mouse button
                                                      #button 1 is left mouse button
                           minspanx=5, minspany=5,    # don't accept a box of fewer than 5 pixels
                           spancoords='pixels' )      # units for above

# to detect the 'R' key press to reset the image
plt.connect("key_press_event", onKeyPressed);

In [189]:
sel_inx = index[0]

selected = app.data[sel_inx]
print(len(sel_inx))
selected['x']

1443


array([-57.2011, -49.2731, -59.6344, ..., -57.7633, -60.6629, -63.7627],
      dtype=float32)

x:  1211 [-39.6964 -57.2011 -49.2731 ... -60.6629 -60.6496 -63.7627]


In [129]:
x,y,z = xyz[:,sel_inx]
vx,vy,vz = vxyz[:,sel_inx]

fig = plt.figure(figsize=(10,10))
fontsize = 16

xlim = ylim = 150

# top left
ax0 = fig.add_subplot(1, 1, 1)
ax0.quiver(x, y, vx, vy, color='gray')

# ax0.set_xlim(-xlim, xlim)
# ax0.set_ylim(-ylim, ylim)

# Add axis labels (standard pyplot)
ax0.set_xlabel('x (kpc)', fontsize=fontsize)
ax0.set_ylabel('y (kpc)', fontsize=fontsize)


Text(0, 0.5, 'y (kpc)')

In [149]:
ax0.patches

In [222]:
?plt.scatter